(nuclio-real-time-functions)=
# Nuclio real-time functions

Nuclio is a high-performance "serverless" framework focused on data, I/O, and compute intensive workloads. It is well integrated with popular 
data science tools, such as Jupyter and Kubeflow; supports a variety of data and streaming sources; and supports execution over CPUs and GPUs. 

You can use Nuclio through a fully managed application service (in the cloud or on-prem) in the Iguazio MLOps Platform. MLRun serving 
utilizes serverless Nuclio functions to create multi-stage real-time pipelines. 

The underlying Nuclio serverless engine uses a high-performance parallel processing engine that maximizes the utilization of CPUs and GPUs, 
supports 13 protocols and invocation methods (for example, HTTP, Cron, Kafka, Kinesis), and includes dynamic auto-scaling for HTTP and 
streaming. Nuclio and MLRun support the full life cycle, including auto-generation of micro-services, APIs, load-balancing, logging, 
monitoring, and configuration management—such that developers can focus on code, and deploy to production faster with minimal work.

Nuclio is extremely fast: a single function instance can process hundreds of thousands of HTTP requests or data records per second. To learn 
more about how Nuclio works, see the Nuclio architecture [documentation](https://nuclio.io/docs/latest/concepts/architecture/). 

Nuclio is secure: Nuclio is integrated with Kaniko to allow a secure and production-ready way of building Docker images at run time.

Read more in the [Nuclio documentation](https://nuclio.io/docs/latest/) and the open-source [MLRun library](https://github.com/mlrun/mlrun).

## Example of Nuclio function

You can create your own Nuclio function, for example a data processing function. For every Nuclio function, by default, there is one worker. See [Number of GPUs](../runtimes/configuring-job-resources.html#number-of-gpus).

The following code illustrates an example of an MLRun function, of kind 'nuclio', that can be deployed to the cluster.

Create a file `func.py` with the code of the function: 
```
def handler(context, event):
    return "Hello"
```    

Create the project and the Nuclio function:

In [ ]:
import mlrun

In [ ]:
# Create the project
project = mlrun.get_or_create_project("nuclio-project", "./")

In [ ]:
# Create a Nuclio function
project.set_function(
    func="func.py",
    image="mlrun/mlrun",
    kind="nuclio",
    name="nuclio-func",
    handler="handler",
)
# Save the function within the project
project.save()
# Deploy the function in the cluster
project.deploy_function("nuclio-func")

## Nuclio API gateway

API gateway is an entity that Nuclio creates on top of one or two functions. From a Kubernetes perspective, an API gateway is just an ingress that links to the Nuclio function. If API gateway is created on top of two functions, it's a {ref}`canary functions<canary>`. The other feature that API gateways provide is authorization. Read more about API gateways in the [Nuclio documentation](https://docs.nuclio.io/en/latest/reference/api-gateway/index.html).
```
{admonition}Note
Since API Gateway is only a Nuclio feature, it is supported only for Nuclio runtimes.
```
This example demonstrates making an HTTP request to an HTTPS API Gateway of a Nuclio function using basic/access key authentication.

In [ ]:
import mlrun
import nuclio

In [ ]:
# Create a project
project = mlrun.get_or_create_project(
    "nuclio-api-gateway-example", context="./", user_project=True
)

In [ ]:
# mlrun: start-code

In [ ]:
def handler(context, event):
    return "test"

In [ ]:
# mlrun: end-code

In [ ]:
# Create a simple Nuclio function that gets basic authentication
basic_auth = project.set_function(
    name="basic-auth", handler="handler", image="mlrun/mlrun", kind="nuclio"
)
# Create a simple nuclio function that gets accesss key authentication
access_key_auth = project.set_function(
    name="acces-key", handler="handler", image="mlrun/mlrun", kind="nuclio"
)
project.save()

In [ ]:
# Deploy the function
basic_auth.deploy()
access_key_auth.deploy()

### Making an HTTP request using basic authentication
1. Create an API Gateway in the UI, with authentication `basic`. Set your desired username and password and choose the `basic-auth` nuclio function.
2. Give it a name and copy the endpoint.
2. Paste the endpoint after the `https://`.
2. Change the username and password in the code below.

In [ ]:
import requests
from base64 import b64encode


# Authorization token: Encode to Base64 format
# and then decode it to ASCII since python 3 stores it as a byte string
def basic_auth(username, password):
    token = b64encode(f"{username}:{password}".encode("utf-8")).decode("ascii")
    return f"Basic {token}"


# Enter your username and password here
username = "username"
password = "password"

# Enter your API Gateway endpoint here
basic_auth_api_gateway_path = "https://<API GATEWAY ENDPOINT>"

headers = {"Authorization": basic_auth(username, password)}
res = requests.get(url=basic_auth_api_gateway_path, headers=headers, verify=False)
print(res.text)

### Making an HTTP request using access key authentication
1. Create an API Gateway in the UI, with authentication `access key` and choose the `access-key` Nuclio function.
2. Give it a name and copy the endpoint.
2. Paste the endpoint after the `https://`.
2. In the UI, click the user's top right icon, then copy the access key from there.
2. Change the access key in the code below.

In [ ]:
# Enter your access key here
access_key = "some-access-key"

# Enter your API Gateway endpoint here
access_key_auth_api_gateway_path = "https://<API GATEWAY ENDPOINT>"

headers = {"Cookie": 'session=j:{"sid": "' + access_key + '"}'}
res = requests.get(url=access_key_auth_api_gateway_path, headers=headers, verify=False)
print(res.text)